In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import json

from nemo.util import parse_colmap_point_cloud, elevation_function
from nemo.plotting import plot_surface, plot_path_3d, plot_3d_points

%load_ext autoreload
%autoreload 2

## Planning based on COLMAP point cloud reconstruction

Load COLMAP points

In [ ]:
path = '../data/redrocks/colmap_points3D.txt'
points, colors = parse_colmap_point_cloud(path)

In [ ]:
# Extract the points within range xyz in [-5, 5]
# points = points[(points[:,0] > -5) & (points[:,0] < 5)]
# points = points[(points[:,1] > -5) & (points[:,1] < 5)]
# points = points[(points[:,2] > -5) & (points[:,2] < 5)]

fig = plot_3d_points(x=points[:,0], y=points[:,1], z=points[:,2], markersize=1)
fig.show()

In [ ]:
points.shape

In [ ]:
# # For Red Rocks
# manual_R = np.array([[-0.25241505,  0.96618594, -0.0526439],
#                      [-0.69407693, -0.21869781, -0.68587789],
#                      [-0.67419868, -0.13658698,  0.72580999]])
# dataparser_T = np.array([[0.0, 1.0, 0.0, 0.02047962136566639],
#                          [1.0, 0.0, 0.0, -0.17118817567825317],
#                          [0.0, 0.0, -1.0, 0.10579380393028259]])
# dataparser_scale = 0.21856701094642245

# For AirSimMountains
manual_R = np.eye(3)
dataparser_transforms = json.load(open('../models/AirSimMountains/dataparser_transforms.json'))
dataparser_T = np.array(dataparser_transforms['transform'])
dataparser_scale = dataparser_transforms['scale']

In [ ]:
points_tf = points @ manual_R.T
points_tf = points_tf @ dataparser_T[:3,:3].T
points_tf = points_tf + dataparser_T[:3,3]
points_tf = points_tf * dataparser_scale

# Swap x and z axes
points_tf = points_tf @ np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]])

In [ ]:
fig = plot_3d_points(x=points_tf[:,0], y=points_tf[:,1], z=points_tf[:,2], color=points_tf[:,2], markersize=1)
fig.show()

Generate DEM

In [ ]:
from scipy.spatial import KDTree

kd_tree = KDTree(points_tf[:,:2])

N_GRID = 100
K = 3
xmin, ymin, zmin = np.min(points_tf, axis=0)
xmax, ymax, zmax = np.max(points_tf, axis=0)
X, Y = np.meshgrid(np.linspace(xmin, xmax, N_GRID), np.linspace(ymin, ymax, N_GRID))

# Sample kd-tree at grid points
Z = np.zeros_like(X)
for i, j in [(i, j) for i in range(N_GRID) for j in range(N_GRID)]:
    # k=1 nearest neighbor
    d, idx = kd_tree.query([X[i,j], Y[i,j]], k=K)
    if K == 1:
        Z[i,j] = points_tf[idx][2]
    else:
        for point in points_tf[idx]:
            Z[i,j] += point[2]
        Z[i,j] /= K

In [ ]:
fig = plot_surface(X, Y, Z)
fig.show()

Generate mesh